## 5.3.- Servidor WEB

En esta seccion se describira como implemetar un servidor web en el SOC ESP8266 que permita, en este caso prender o apagar un led. Se utilizara una libreria especialmente desarrollada con este fin.
### 5.3.1 Librería ESP8266WebServer
 
Un servidor web es una computadora que almacena los archivos que componen un sitio web (ej.  documentos HTML , imágenes, hojas de estilos CSS y archivo JavaScript) y los entrega a otro dispositivo que le solicita un servicio. Se desarrollo para el ESP8266 una libreria que permite implementar un servidor web. Se denomina libreria esp8266WebServer. Los metodos que se utilizan se muestran a continuacion



| Metodo          | Sintaxis                   |  Funcion      |
|---              |---                         |---            |
|ESP8266WebServer |ESP8266WebServer server(80) | Crea el objeto server |
|begin            | server.begin()             | Inicializa el servidor|
|close            | server.close()             | Apaga el servidor     |
|stop             | server.stop()              | Apaga el servidor     |
| on              | server.on("/", FuncionM);  | Indica una función que emitira una respuesta del servidor cuando un navegador solicita la URL |
|addHandler       | server. addHandler();      |              |
| onNotFound      | server.onNotFound();       |              |
| onFileUpload    | server.onFileUpload();     |              |
| send            | server.send(code,tipo_contenido,contenido)| Envia las respuestas solicitadas code:200, 404 tipo_contenido:"text/plain","image/png"|
|streamFile      |server.streamFile(file, GetContentType(path));| Permite cargar una pagina completa |
|handleClient     |server.handleClient();      |  Maneja las conexiones entrantes|


### 5.3.2 PROGMEM

Permite cargar informacion en la memoria flash. Despues se puede pasar a la RAM cuando hace falta. Es una característica Arduino AVR que ha sido portada a ESP8266 para asegurar la compatibilidad con las librerías existentes en Arduino, así como para ahorrar RAM. La sintaxis seria:

``` const char MAIN_page[] PROGMEM = R"=====( pagina web )======" ```


### 5.3.3 Programa Webserver
Este programa debe prender o apagar el led que se encuentra en la placa desde una pagina WEB. Este sistema puede ser controlado desde un telefono celular. Se utiliza el microcontrolador esp8266 que se conecta a la red WIFI y crea un webserver. Utliza la memoria RAM para guardar la pagina WEB.

Se crea un nuevo programa en el IDE de arduino denominado internet1.ino y los archivos config.h y wifi.hpp se copian en el nuevo directorio.
